# 股票数据

We need statically-storaged stock data

In [1]:
import requests
import re

In [10]:
import traceback
import tqdm

经查新浪财经是动态数据，百度财经接口不稳定，这里选取腾讯财经。腾讯财经是静态数据且接口稳

至于股票代码列表，东方财富网原链接也不见了，新的行情中心采用的是动态数据，这里使用一个小网站的静态数据。

In [2]:
stock_list_url = 'https://quote.stockstar.com/stock/stock_index.htm'

#for example https://gu.qq.com/sz000001/
stock_info_url = 'https://gu.qq.com/' 
#stock exchange code 
secode = 'sz'
#stock code
stockcode = '000001'
#code
scode = secode + stockcode


In [3]:
def getHTMLText(url, headers_config, offline_encoding=True):
    try:
        r = requests.get(url, timeout=10, headers = headers_config)
        r.raise_for_status()
        if offline_encoding:
            r.encoding='utf-8'
        else:
            r.encoding=r.apparent_encoding
        return r.text
    except:
        traceback.print_exc()
        return ""

In [4]:
html = getHTMLText(stock_info_url+scode, None)

In [5]:
print(html)


    <!DOCTYPE html>
    <html>
      <head>
				<title></title>
				<meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=11; IE=10; IE=9; IE=8; IE=edge;">
        <link rel="dns-prefetch" href="//mat1.gtimg.com">
        <link rel="dns-prefetch" href="//proxy.finance.qq.com/ifzqgtimg">
        <link rel="dns-prefetch" href="//qt.gtimg.cn">
        <meta name="renderer" content="webkit">
        <!--[if lt IE 8]><script>window.location.href='//finance.qq.com/gsfinance/upgrade_browser.htm'</script><![endif]-->
		<meta name="react-comment-hack" >
			<!--[if lte IE 9]><script src="//mat1.gtimg.com/finance/js/ie/es5-shim.min.js"></script><script src="//mat1.gtimg.com/finance/js/ie/es5-sham.js"></script><![endif]-->
			<!--[if IE]><script src="//mat1.gtimg.com/finance/js/flotr2/ie.js"></script><![endif]-->
        </meta>
        <script>
        	var QosSS = [+new Date()];
       		QosSS[4] = QosSS[3] = QosSS[2] = QosSS[1] = QosSS[0];
        </script>
        <scr

分析网页源码知，价格存储在一个span标签内，且无特征字符串（相比前面淘宝价格的例子而言）。因而单纯从正则表达式出发匹配不易。这里仍然使用bs库解析标签树。

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(html, 'html.parser')

In [8]:
soup.find(name='span', attrs='data').string

'14.92'

这正是平安银行当天的股价

接下来是提取股票代码列表

In [13]:
html_list = getHTMLText(stock_list_url, None)

Traceback (most recent call last):
  File "d:\Anaconda\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "d:\Anaconda\lib\site-packages\urllib3\connectionpool.py", line 1040, in _validate_conn
    conn.connect()
  File "d:\Anaconda\lib\site-packages\urllib3\connection.py", line 426, in connect
    tls_in_tls=tls_in_tls,
  File "d:\Anaconda\lib\site-packages\urllib3\util\ssl_.py", line 450, in ssl_wrap_socket
    sock, context, tls_in_tls, server_hostname=server_hostname
  File "d:\Anaconda\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "d:\Anaconda\lib\ssl.py", line 423, in wrap_socket
    session=session
  File "d:\Anaconda\lib\ssl.py", line 870, in _create
    self.do_handshake()
  File "d:\Anaconda\lib\ssl.py", line 1139, in do_handshake
    self._sslobj.do_handshake()
socket.timeout: _ssl.c:1059: The handshake oper

In [17]:
soup_list = BeautifulSoup(html_list, 'html.parser')

In [18]:
li = soup_list.find_all('li')

In [19]:
li[:10]

[<li class="seo_current" id="index_parent_0"><a href="javascript:void('0')" onclick="changeIndexTab(0,4)" target="_self">��A</a></li>,
 <li id="index_parent_1"><a href="javascript:void('0')" onclick="changeIndexTab(1,4)" target="_self">��B</a></li>,
 <li id="index_parent_2"> <a href="javascript:void('0')" onclick="changeIndexTab(2,4)" target="_self">��A</a></li>,
 <li id="index_parent_3"><a href="javascript:void('0')" onclick="changeIndexTab(3,4)" target="_self">��B</a></li>,
 <li><span><a href="/gs/sh_600000.shtml">600000</a></span><a href="//stock.quote.stockstar.com/600000.shtml">�ַ�����</a></li>,
 <li><span><a href="/gs/sh_600001.shtml">600001</a></span><a href="//stock.quote.stockstar.com/600001.shtml">��������</a></li>,
 <li><span><a href="/gs/sh_600002.shtml">600002</a></span><a href="//stock.quote.stockstar.com/600002.shtml">��³ʯ��</a></li>,
 <li><span><a href="/gs/sh_600003.shtml">600003</a></span><a href="//stock.quote.stockstar.com/600003.shtml">ST������</a></li>,
 <li><span

In [20]:
li[5].find_all('a')[0].string

'600001'

In [21]:
scode_list = []
sname_list = []
for i in li:
    try:
        a = i.find_all('a')
        scode = a[0].string
        sname = a[1].string
        scode_list.append(scode)
        sname_list.append(sname)
    except:
        continue

封装

In [22]:
def getStockList(stock_list_url, offline_loading = ''):
    html_list = getHTMLText(stock_list_url, None)

    #in case of out of time
    if html_list == '' or html_list is None:
        html_list = offline_loading
    soup_list = BeautifulSoup(html_list, 'html.parser')
    li = soup_list.find_all('li')
    scode_dict = {}
    for i in li:
        try:
            a = i.find_all('a')
            scode = a[0].string
            sname = a[1].string
            scode_dict[scode] = sname
        except:
            continue
    
    return scode_dict

In [27]:
def getStockInfo(scode_dict, stock_info_url, fpath = './stock_info.csv'):
    secode =''
    price = '0'
    with open(fpath, 'w', encoding='utf-8') as csv:
        with tqdm(total=len(scode_dict.items())) as progress_bar:
            for scode, sname in scode_dict.items():
                try:
                    prefix = scode[0]
                    if prefix in ['6','9']:
                        secode = 'sh'
                    elif prefix in ['0','2']:
                        secode = 'sz'
                    else:
                        ValueError('unknown code')
                        break
                    
                    url = stock_info_url + secode + scode
                    html = getHTMLText(url, None)
                    soup = BeautifulSoup(html, 'html.parser')
                    price = soup.find(name='span', attrs='data').string
                    csv.write("%s,%s,%s\n" %(scode, sname, price))
                    progress_bar.update(1)
                except:
                    traceback.print_exc()
                    continue
                    
    csv.close()


In [35]:
r = requests.get(stock_list_url)

In [33]:
scode_dict = getStockList(stock_list_url=stock_list_url, offline_loading=False)

In [34]:
scode_dict

{'600000': '�ַ�����',
 '600001': '��������',
 '600002': '��³ʯ��',
 '600003': 'ST������',
 '600004': '���ƻ���',
 '600005': '��ֹɷ�',
 '600006': '��������',
 '600007': '�й���ó',
 '600008': '�״�����',
 '600009': '�Ϻ�����',
 '600010': '���ֹɷ�',
 '600011': '���ܹ���',
 '600012': '��ͨ����',
 '600015': '��������',
 '600016': '��������',
 '600017': '���ո�',
 '600018': '�ϸۼ���',
 '600019': '���ֹɷ�',
 '600020': '��ԭ����',
 '600021': '�Ϻ�����',
 '600022': 'ɽ������',
 '600023': '���ܵ���',
 '600025': '����ˮ��',
 '600026': '��Զ����',
 '600027': '�������',
 '600028': '�й�ʯ��',
 '600029': '�Ϸ�����',
 '600030': '����֤ȯ',
 '600031': '��һ�ع�',
 '600032': '�㽭����',
 '600033': '��������',
 '600035': '�������',
 '600036': '��������',
 '600037': '�軪����',
 '600038': '��ֱ�ɷ�',
 '600039': '�Ĵ�·��',
 '600048': '������չ',
 '600050': '�й���ͨ',
 '600051': '��������',
 '600052': '����ʱ��',
 '600053': '�Ŷ�Ͷ��',
 '600054': '��ɽ����',
 '600055': '��ҽ��',
 '600056': '�й�ҽҩ',
 '600057': '��������',
 '600058': '���չ',
 '60

In [99]:
getStockInfo(scode_dict, stock_info_url)

utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8


Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_15420\1993853575.py", line 19, in getStockInfo
    price = soup.find(name='span', attrs='data').string
AttributeError: 'NoneType' object has no attribute 'string'


utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8


Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_15420\1993853575.py", line 19, in getStockInfo
    price = soup.find(name='span', attrs='data').string
AttributeError: 'NoneType' object has no attribute 'string'


utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8


Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_15420\1993853575.py", line 19, in getStockInfo
    price = soup.find(name='span', attrs='data').string
AttributeError: 'NoneType' object has no attribute 'string'


utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8


: 

In [77]:
with open('./stock_info.csv', 'w') as csv:
    for scode, sname in scode_dict.items():
        prefix = scode[0]
        if prefix in ['6','9']:
            secode = 'sh'
        elif prefix in ['0','2']:
            secode = 'sz'
        else:
            ValueError('unknown code')
            break
        
        url = stock_info_url + secode + scode
        html = getHTMLText(url, None)
        soup = BeautifulSoup(html, 'html.parser')
        price = soup.find(name='span', attrs='data').string
        print(price)
        csv.write("%s,%s,%s\n" %(scode, sname, price))
csv.close()

In [80]:
secode

'sz'

In [82]:
scode

'600000'

In [89]:
scode_list[411]

'600487'

弃疗了，网络连接不稳定，我想改encoding的时候小网站又打不开了。反正总体功能已经实现了，但是效率奇低，鸡肋。加上本来对股票毫不感冒，直接跳过